In [37]:
import mne
import math
import numpy as np
import pickle
import random

from os.path import exists
from sklearn import preprocessing


# Important Parameters to Set

In [38]:
Sampling_Frequency=250
Length_of_trail=30
Number_of_FFT_Features_To_Select=100
healthy=14
schizo=14
file_name=['h','s']
Band_Pass_Low_Range=0
Band_Pass_High_Range=50
indices = list(range(1, healthy + 1))
random.shuffle(indices)
indices

In [39]:
def select_channel(raw,selected_channel=['Fp1','Fp2']):
    channel_indices=[]
    for channel in selected_channel:
        try:
            index=raw.ch_names.index(channel)
            channel_indices.append(index)
        except ValueError:
            print('Channel{} is not present or again check the names of channel'.format())
    new_raw=raw.get_data(picks=channel_indices)
    return new_raw
            


In [40]:
def select_features(new_raw,label,samp_freq=Sampling_Frequency,recording_time=Length_of_trail):     # Enter Raw Data, Label you want to give, Sampling Frequency and Recording Time 
    trail_length=samp_freq*recording_time   # Calculating Length of each Trail
    trail_length_shape=np.shape(new_raw)[1]   # Determining the time of each channel
    total_chunks=trail_length_shape// trail_length    # Final How many chunks can be created 
    new_raw=new_raw[:,0:total_chunks*trail_length]     # Changing Shape of Data
    chunks_raw=np.split(new_raw,total_chunks,axis=1)    # Creating the trails
    # Now applying Fourier Transform on each Chunk
    feature=[]
    for chunk in chunks_raw:
        segg=[]
        for i in range(chunk.shape[0]):
            fourier=np.abs(np.fft.fft(chunk[i,:]))[:Number_of_FFT_Features_To_Select]
            segg.append(fourier)
        final_concate=np.concatenate(segg,axis=0)
        feature.append((final_concate,label))
    return feature   # Returning tuple which contain the feature obtained of each trail through FFT and the corresponding Label of each trail Feature

    
    
    

In [41]:
def Preprocessing(folder_name,indicess,labels,train_percentage=0.8,Band_Pass_Low_Range=0,Band_Pass_High_Range=50,number_of_subjects=14):
    train_current_seq=1
    class_train=[]
    class_test=[]
    train_limit = math.ceil(train_percentage * number_of_subjects)
    for i in indicess:
        if exists('./Dataset/{}{}.edf'.format(folder_name,i)):
            file='./Dataset/{}{}.edf'.format(folder_name,i)
            raw=mne.io.read_raw_edf(file,preload=True,verbose=0,eog=None,misc=None,stim_channel='auto',exclude=())
            raw.pick_types(eeg=True)
            raw.filter(Band_Pass_Low_Range,Band_Pass_High_Range)
            new_raw=select_channel(raw)
            features=select_features(new_raw,label=labels)
            if train_current_seq <=train_limit:
                class_train+=features
            else:
                class_test+=features
            train_current_seq=train_current_seq+1
    return class_train,class_test

<>:7: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\S'
<>:7: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20548\2629244717.py:7: SyntaxWarning: invalid escape sequence '\S'
  if exists('D:\Schzo\Dataset\Dataset\{}{}.edf'.format(folder_name,i)):
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20548\2629244717.py:8: SyntaxWarning: invalid escape sequence '\S'
  file='D:\Schzo\Dataset\Dataset\{}{}.edf'.format(folder_name,i)


In [42]:
def Equal_length(pos_class_train,neg_class_train):
    if len(pos_class_train) > len(neg_class_train):
        pos_class_train = pos_class_train[:len(neg_class_train)]
    elif len(pos_class_train) < len(neg_class_train):
        neg_class_train = neg_class_train[:len(pos_class_train)]
    return pos_class_train,neg_class_train



In [43]:
pos_class_train,pos_class_test=Preprocessing(folder_name='h',indicess=indices,Band_Pass_Low_Range=0,Band_Pass_High_Range=50,labels=0,train_percentage=0.7)
neg_class_train,neg_class_test=Preprocessing(folder_name='s',indicess=indices,Band_Pass_Low_Range=0,Band_Pass_High_Range=50,labels=1,train_percentage=0.7)
pos_class_train,neg_class_train=Equal_length(pos_class_train,neg_class_train)
pos_class_test,neg_class_test=Equal_length(pos_class_test,neg_class_test)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 67 samples (0.268 s)



NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 67 samples (0.268 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 67 samples (0.268 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 67 samples (0.268 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 67 samples (0.268 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [44]:
Training_data=pos_class_train+neg_class_train
Testing_data=pos_class_test+neg_class_test

In [45]:
# Using double backslashes
with open('./samples/training.pkl', 'wb') as f:
    pickle.dump(Training_data, f)

with open('./samples/testing.pkl', 'wb') as f:
    pickle.dump(Testing_data, f)